In [2]:
import csv

In [3]:
with open('./data/converted_crime_data/Crime_pass2_strip_bottomhalf.csv') as f:
    data = [row for row in csv.reader(f)]

In [4]:
list(enumerate(data[0]))

[(0, 'ADDRESS'),
 (1, 'Apt'),
 (2, 'Caee'),
 (3, 'Primary Offense'),
 (4, 'Off_Code'),
 (5, 'Offense'),
 (6, 'tract'),
 (7, 'repdate'),
 (8, 'date_rept'),
 (9, 'date_occu'),
 (10, 'hour_occu'),
 (11, 'dow1'),
 (12, 'date_fnd'),
 (13, 'hour_fnd'),
 (14, ''),
 (15, ''),
 (16, ''),
 (17, ''),
 (18, '')]

In [51]:
from collections import Counter, defaultdict
print(Counter(sum(x == '' for x in row[14:]) for row in data))

counts = defaultdict(int)
for row in data:
    for i, x in enumerate(row[14:]):
        if x == '':
            counts[i+14] += 1
print(counts)

Counter({5: 1735, 2: 1381})
defaultdict(<class 'int'>, {16: 1735, 17: 3116, 18: 3116, 14: 1735, 15: 1735})


so over half of the rows don't have address data...this is a property of the intial data set

In [94]:
from random import sample
def get_sampled_row(row_index, data=data, sample_size=30):
    return [data[0][row_index]] + [x[row_index] for x in sample(data[1:], sample_size)]

In [39]:
get_sampled_row(9, sample_size=50)
Counter(int(row[9] != '') for row in data[1:])

Counter({0: 1103, 1: 2012})

so about 1/3 are empty...in 2013 there's a shift of date_occ -> hour_occ and hour_occ -> dow1

In [55]:
get_sampled_row(14, sample_size=10)

['', '16', '', '', '70', '848', '73', '', '', '884', '']

so rows 17 and 18 are garbage data universally
row 14 is still FID

let's split up into two datasets for the two schemas
let's find out if there are runs on column 9 that enable a clean easy split
speculate that the year 2013 starts at data[i=2013] from the Counter result

In [62]:
all(row[9] != ''  for row in data[1:2013]) and all(row[9] == '' for row in data[2013:])

True

In [86]:
data_a = [data[0]] + data[1:2013]
data_b = [data[0]] + data[2013:]

delete rows 17 and 18...easy since len(row) == 19

In [87]:
data_a = [row[:17] for row in data_a]
data_b = [row[:17] for row in data_b]

# data_a

In [88]:
all(row[7] == '' for row in data_a[1:])

True

In [90]:
list(enumerate(zip(data_a[0], data_a[1])))

[(0, ('ADDRESS', '1520 LIGURIA AV')),
 (1, ('Apt', '')),
 (2, ('Caee', '11000040')),
 (3, ('Primary Offense', '1')),
 (4, ('Off_Code', 'BUA')),
 (5, ('Offense', 'BURGLARY RESIDENTIAL')),
 (6, ('tract', '07U')),
 (7, ('repdate', '')),
 (8, ('date_rept', '01/ 3/2011')),
 (9, ('date_occu', '11/01/03')),
 (10, ('hour_occu', '[$-409]12/23/10 19:00 a12/p12')),
 (11, ('dow1', '[$-409]19:00 a12/p12')),
 (12, ('date_fnd', '[$-409]12/28/10 22:00 a12/p12')),
 (13, ('hour_fnd', '[$-409]22:00 a12/p12')),
 (14, ('', '385')),
 (15, ('', '891902.84400000004')),
 (16, ('', '502168.55800000002'))]

throw out repdate (i=7) and hour_occu (i=10) and use dow1 as hour_occu

In [91]:
data_a = [
    row[:7] + row[8:10] + row[11:]
    for row in data_a[1:]
]

In [84]:
data_a_key = [
    'address', 'apt',
    'case', 'primary_offense', 'offense_code', 'offense', 'tract',
    'date_rept',
    'date_occu', 'hour_occu', 
    'date_fnd', 'hour_fnd',
    'fid', 'x', 'y',
]

In [93]:
list(enumerate(zip(data_a_key, data_a[0])))

[(0, ('address', '1520 LIGURIA AV')),
 (1, ('apt', '')),
 (2, ('case', '11000040')),
 (3, ('primary_offense', '1')),
 (4, ('offense_code', 'BUA')),
 (5, ('offense', 'BURGLARY RESIDENTIAL')),
 (6, ('tract', '07U')),
 (7, ('date_rept', '01/ 3/2011')),
 (8, ('date_occu', '11/01/03')),
 (9, ('hour_occu', '[$-409]19:00 a12/p12')),
 (10, ('date_fnd', '[$-409]12/28/10 22:00 a12/p12')),
 (11, ('hour_fnd', '[$-409]22:00 a12/p12')),
 (12, ('fid', '385')),
 (13, ('x', '891902.84400000004')),
 (14, ('y', '502168.55800000002'))]

In [139]:
get_sampled_row(10, data=data_a, sample_size=30)

['[$-409]12/28/10 22:00 a12/p12',
 '03/22/2011',
 '02/21/2011',
 '01/29/2011',
 '02/25/2011',
 '08/27/2012',
 '10/10/2011',
 '01/19/2011',
 '03/23/2011',
 '03/30/2011',
 '10/ 7/2011',
 '09/29/2011',
 '09/ 7/2011',
 '10/29/2012',
 '08/29/2011',
 '12/11/2011',
 '08/27/2012',
 '10/31/2012',
 '04/10/2011',
 '02/ 8/2011',
 '12/ 4/2011',
 '01/19/2011',
 '11/30/2012',
 '08/30/2011',
 '02/20/2011',
 '09/13/2011',
 '10/ 5/2011',
 '12/10/2011',
 '07/17/2012',
 '03/23/2011',
 '02/16/2011']

row 7
    s.replace(' ', '0')
    %m/%d/%Y
row 8
    %y/%m/%d
row 9, 11
    s[6:11].replace(':', '')
row 10
    s.replace('[$-409]', '').replace('/10 ', '/2010 ').replace('/10 ', '/2011 ')[:10].replace(' ', '0')

In [146]:
from datetime import datetime
clean7 = lambda s: datetime.strptime(s.replace(' ', '0'), '%m/%d/%Y')
clean8 = lambda s: datetime.strptime(s, '%y/%m/%d')
clean911 = lambda s: s[7:12].replace(':', '')
clean10 = lambda s: datetime.strptime(
    s.replace('[$-409]', '').replace('/10 ', '/2010 ').replace('/10 ', '/2011 ')[:10].replace(' ', '0'),
    '%m/%d/%Y')

In [147]:
def clean_a_row(row):
    return row[:7] + [
        clean7(row[7]),
        clean8(row[8]),
        clean911(row[9]),
        clean10(row[10]),
        clean911(row[11]),
    ] + row[12:]

In [150]:
list(enumerate(zip(data_a[666], clean_a_row(data_a[666]))))

[(0, ('5800 SAN AMARO DR', '5800 SAN AMARO DR')),
 (1, ('', '')),
 (2, ('11004756', '11004756')),
 (3, ('2', '2')),
 (4, ('ZOA', 'ZOA')),
 (5, ('TRESPASS WARNING', 'TRESPASS WARNING')),
 (6, ('07U', '07U')),
 (7, ('06/20/2011', datetime.datetime(2011, 6, 20, 0, 0))),
 (8, ('11/06/20', datetime.datetime(2011, 6, 20, 0, 0))),
 (9, ('[$-409]14:00 a06/p06', '1400')),
 (10, ('06/20/2011', datetime.datetime(2011, 6, 20, 0, 0))),
 (11, ('[$-409]10:00 a06/p06', '1000')),
 (12, ('586', '586')),
 (13, ('891990.73199999996', '891990.73199999996')),
 (14, ('503116.70500000002', '503116.70500000002'))]

In [151]:
data_a_clean = [clean_a_row(row) for row in data_a]

# data_b

In [153]:
list(enumerate(zip(data_b[0], data_b[1])))

[(0, ('ADDRESS', '5030 BRUNSON DR')),
 (1, ('Apt', '')),
 (2, ('Caee', '13000129')),
 (3, ('Primary Offense', '1')),
 (4, ('Off_Code', 'ZOA')),
 (5, ('Offense', 'TRESPASS WARNING')),
 (6, ('tract', '07U')),
 (7, ('repdate', '')),
 (8, ('date_rept', '01/08/13')),
 (9, ('date_occu', '')),
 (10, ('hour_occu', '01/08/13')),
 (11, ('dow1', '0840')),
 (12, ('date_fnd', '01/08/13')),
 (13, ('hour_fnd', '0844')),
 (14, ('', '')),
 (15, ('', '')),
 (16, ('', ''))]

In [157]:
all(row[7] == '' for row in data_b[1:]) and all(row[9] == '' for row in data_b[1:])

True

throw out i=7, 9 

In [158]:
data_b = [
    row[:7] + row[8:9] + row[10:]
    for row in data_b[1:]
]

In [160]:
data_b_key = [
    'address', 'apt',
    'case', 'primary_offense', 'offense_code', 'offense', 'tract',
    'date_rept',
    'date_occu', 'hour_occu', 
    'date_fnd', 'hour_fnd',
    'fid', 'x', 'y',
]

In [161]:
list(enumerate(zip(data_b_key, data_b[0])))

[(0, ('address', '5030 BRUNSON DR')),
 (1, ('apt', '')),
 (2, ('case', '13000129')),
 (3, ('primary_offense', '1')),
 (4, ('offense_code', 'ZOA')),
 (5, ('offense', 'TRESPASS WARNING')),
 (6, ('tract', '07U')),
 (7, ('date_rept', '01/08/13')),
 (8, ('date_occu', '01/08/13')),
 (9, ('hour_occu', '0840')),
 (10, ('date_fnd', '01/08/13')),
 (11, ('hour_fnd', '0844')),
 (12, ('fid', '')),
 (13, ('x', '')),
 (14, ('y', ''))]

In [176]:
get_sampled_row(11, data=data_b)

['0844',
 '0100',
 '0158',
 '1621',
 '2359',
 '1351',
 '1825',
 '2102',
 '0733',
 '1639',
 '1130',
 '0900',
 '2030',
 '2249',
 '0900',
 '0850',
 '1712',
 '0445',
 '1240',
 '1030',
 '1630',
 '1630',
 '1745',
 '1010',
 '0800',
 '0713',
 '0814',
 '0745',
 '1000',
 '2355',
 '1220']

row7,8,10
    %m/%d/%y

In [183]:
clean7810 = lambda s: datetime.strptime(s, '%m/%d/%y')

In [179]:
def clean_b_row(row):
    return row[:7] + [
        clean7810(row[7]),
        clean7810(row[8]),
        row[9],
        clean7810(row[10]),
    ] + row[11:]

In [184]:
data_b_clean = [clean_b_row(row) for row in data_b]

In [186]:
list(enumerate(zip(data_b[66], data_b_clean[66])))

[(0, ('6200 SAN AMARO DR', '6200 SAN AMARO DR')),
 (1, ('', '')),
 (2, ('13000599', '13000599')),
 (3, ('1', '1')),
 (4, ('ZOB', 'ZOB')),
 (5, ('WARRANT ARREST', 'WARRANT ARREST')),
 (6, ('07U', '07U')),
 (7, ('01/25/13', datetime.datetime(2013, 1, 25, 0, 0))),
 (8, ('01/24/13', datetime.datetime(2013, 1, 24, 0, 0))),
 (9, ('2230', '2230')),
 (10, ('01/24/13', datetime.datetime(2013, 1, 24, 0, 0))),
 (11, ('2235', '2235')),
 (12, ('', '')),
 (13, ('', '')),
 (14, ('', ''))]

# union data

In [187]:
data_union = [data_a_key] + data_a_clean + data_b_clean

In [188]:
data_union[:2]

[['address',
  'apt',
  'case',
  'primary_offense',
  'offense_code',
  'offense',
  'tract',
  'date_rept',
  'date_occu',
  'hour_occu',
  'date_fnd',
  'hour_fnd',
  'fid',
  'x',
  'y'],
 ['1520 LIGURIA AV',
  '',
  '11000040',
  '1',
  'BUA',
  'BURGLARY RESIDENTIAL',
  '07U',
  datetime.datetime(2011, 1, 3, 0, 0),
  datetime.datetime(2011, 1, 3, 0, 0),
  '1900',
  datetime.datetime(2010, 12, 28, 0, 0),
  '2200',
  '385',
  '891902.84400000004',
  '502168.55800000002']]

# sanity check

In [189]:
from datetime import datetime

In [190]:
weird_future_rows = [
    (i, row) for i, row in enumerate(data_union[1:])
    if row[8].year > 2013
]

In [191]:
len(weird_future_rows)

0

In [192]:
weird_old_rows = [
    (i, row) for i, row in enumerate(data_union[1:])
    if row[8].year < 2010
]

In [193]:
len(weird_old_rows)

1

In [194]:
weird_old_rows

[(2002,
  ['1239 DICKINSON DR',
   'WT207',
   '12009104',
   '1',
   'ZOQ',
   'ULEAD',
   '07U',
   datetime.datetime(2012, 11, 16, 0, 0),
   datetime.datetime(2002, 11, 16, 0, 0),
   '0145',
   datetime.datetime(2012, 11, 16, 0, 0),
   '0205',
   '74',
   '892974.25199999998',
   '503706.15500000003'])]

manually correct this typo

In [199]:
data_union[2003][8] = datetime(2012, 11, 16)
data_union[2003][8]

datetime.datetime(2012, 11, 16, 0, 0)

In [200]:
from datetime import timedelta
def probable_misentry(a, b, months=2, check_same_day=True):
    reasonable_entry_lag = timedelta(weeks=months*4)
    
    if abs(a-b) > reasonable_entry_lag:
        if not check_same_day:
            return True
        return a.month == b.month and a.day == b.day
    return False

# TODO could manually correct them
data_outlier_remove = [data_union[0]] + [
    row for row in data_union[1:]
    if not probable_misentry(row[7], row[8], check_same_day=False)
]

print('removed {} points'.format(len(data_union)-len(data_outlier_remove)))

removed 20 points


In [201]:
from collections import defaultdict

In [202]:
yearly_data = defaultdict(list)
for row in data_outlier_remove[1:]:
    yearly_data[row[8].year].append(row)

In [203]:
yearly_data.keys()

dict_keys([2011, 2012, 2013])

now, let's write a separate file for each year and make sure to only take date for date columns

In [204]:
list(enumerate(data_outlier_remove[0]))

[(0, 'address'),
 (1, 'apt'),
 (2, 'case'),
 (3, 'primary_offense'),
 (4, 'offense_code'),
 (5, 'offense'),
 (6, 'tract'),
 (7, 'date_rept'),
 (8, 'date_occu'),
 (9, 'hour_occu'),
 (10, 'date_fnd'),
 (11, 'hour_fnd'),
 (12, 'fid'),
 (13, 'x'),
 (14, 'y')]

In [205]:
for year, values in yearly_data.items():
    fn = './data/converted_crime_data/Crime_pass3_{}.csv'.format(year)
    print('writing ' + fn)
    with open(fn, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(data_outlier_remove[0])
        for row in values:
            writer.writerow(
                row[:7] + [
                    row[7].strftime("%Y-%m-%d"), row[8].strftime("%Y-%m-%d"),
                    row[9], row[10].strftime("%Y-%m-%d")
                ] + row[11:]
            )

writing ./data/converted_crime_data/Crime_pass3_2011.csv
writing ./data/converted_crime_data/Crime_pass3_2012.csv
writing ./data/converted_crime_data/Crime_pass3_2013.csv
